In [5]:
import random
from collections import Counter, defaultdict
from pymongo import MongoClient
import pickle
import os
import json
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
from utils import *
from sklearn.metrics.pairwise import cosine_similarity
import jieba
import re
import matplotlib.pyplot as plt
from datetime import timedelta

In [2]:
class SysPrediction:
    def __init__(self, db="system_predictions"):
        client = MongoClient('mongodb://127.0.0.1:27017', username='', password="")
        if db == "system_predictions_all":
            self.system_predictions = client.Assess.system_predictions_all
        else:
            self.system_predictions = client.Assess.system_predictions
        self.user_ids = set([t['user_id'] for t in self.system_predictions.find()])
        self.user_model_ranks = {}
        self.user_model_predictions = {}
        self.__build_ranks()
        self.__build_predictions()
    
    def __build_ranks(self):
        self.user_model_ranks = {}
        for tu in self.user_ids:
            upred = {}
            for tpred in self.system_predictions.find({"user_id": tu}):
                upred[tpred['model']] = sorted([t for t in tpred['predictions'] if pd.isnull(t['score']) == False], key=lambda t: t['score'], reverse=True)
            self.user_model_ranks[tu] = upred
    
    def __build_predictions(self):
        self.user_model_predictions = {}
        for tu in self.user_ids:
            upred = {}
            for tpred in self.system_predictions.find({"user_id": tu}):
                upred.setdefault(tpred['model'], {})
                for t in tpred['predictions']:
                    upred[tpred['model']][t['movieId']] = t['score']
            self.user_model_predictions[tu] = upred
    
    def get_prediction(self, user_id, item_id, model):
        if user_id in self.user_model_predictions:
            if model in self.user_model_predictions[user_id]:
                if item_id in self.user_model_predictions[user_id][model]:
                    return self.user_model_predictions[user_id][model][item_id]
        # print ("No prediction", user_id, item_id, model)
        return np.nan

    def get_rank(self, user_id, model, k=10):
        if user_id in self.user_model_ranks:
            if model in self.user_model_ranks[user_id]:
                if k != None:
                    return self.user_model_ranks[user_id][model][:k]
                return self.user_model_ranks[user_id][model]
        print ("No rank", user_id, model)
        return []
    
    def get_user_candidates(self, user_id):
        if user_id in self.user_model_ranks:
            if "random" in self.user_model_predictions[user_id]:
                return [t for t in self.user_model_predictions[user_id]['random'].keys()]
            print ("no random!")

In [4]:
# if __name__ == "__main__":
#     sys_pred = SysPrediction()
#     sys_pred.get_rank("u_13572956735", "pop")